In [12]:
from matplotlib import pyplot as plt
import torch
from torchvision import datasets, transforms
from torch import nn
import torch.nn.functional as F
import numpy as np

torch.manual_seed(1)
np.random.seed(1)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


import pandas as pd 

UNLABELED_BS = 256
TRAIN_BS = 32
TEST_BS = 1024

num_train_samples = 1000
samples_per_class = int(num_train_samples/10)

x = pd.read_csv('data/mnist_train.csv')
y = x['label']
x.drop(['label'], inplace = True, axis = 1)

x_test = pd.read_csv('data/mnist_test.csv')
y_test = x_test['label']
x_test.drop(['label'], inplace = True, axis = 1)

x_train, x_unlabeled = x[y.values == 0].values[:samples_per_class], x[y.values == 0].values[samples_per_class:]
y_train = y[y.values == 0].values[:samples_per_class]

for i in range(1,10):
    x_train = np.concatenate([x_train, x[y.values == i].values[:samples_per_class]], axis = 0)
    y_train = np.concatenate([y_train, y[y.values == i].values[:samples_per_class]], axis = 0)
    
    x_unlabeled = np.concatenate([x_unlabeled, x[y.values == i].values[samples_per_class:]], axis = 0)

from sklearn.preprocessing import Normalizer

normalizer = Normalizer()
x_train = normalizer.fit_transform(x_train)
x_unlabeled = normalizer.transform(x_unlabeled)
x_test = normalizer.transform(x_test.values)
# x_train = x_train / 255.0
# x_test = x_test.values / 255.0


x_train = torch.from_numpy(x_train).type(torch.FloatTensor)
y_train = torch.from_numpy(y_train).type(torch.LongTensor) 

x_test = torch.from_numpy(x_test).type(torch.FloatTensor)
y_test = torch.from_numpy(y_test.values).type(torch.LongTensor)

train = torch.utils.data.TensorDataset(x_train, y_train)
test = torch.utils.data.TensorDataset(x_test, y_test)


In [13]:

train_loader = torch.utils.data.DataLoader(train, batch_size = TRAIN_BS, shuffle = True, num_workers = 8)

unlabeled_train = torch.from_numpy(x_unlabeled).type(torch.FloatTensor)

unlabeled = torch.utils.data.TensorDataset(unlabeled_train)
unlabeled_loader = torch.utils.data.DataLoader(unlabeled, batch_size = UNLABELED_BS, shuffle = True, num_workers = 8)

test_loader = torch.utils.data.DataLoader(test, batch_size = TEST_BS, shuffle = True, num_workers = 8)

# Architecture from : https://github.com/peimengsui/semi_supervised_mnist
class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(1, 20, kernel_size=5)
            self.conv2 = nn.Conv2d(20, 40, kernel_size=5)
            self.conv2_drop = nn.Dropout2d()
            self.fc1 = nn.Linear(640, 150)
            self.fc2 = nn.Linear(150, 10)
            self.log_softmax = nn.LogSoftmax(dim = 1)

        def forward(self, x):
            x = x.view(-1,1,28,28)
            x = F.relu(F.max_pool2d(self.conv1(x), 2))
            x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
            x = x.view(-1, 640)
            x = F.relu(self.fc1(x))
            x = F.dropout(x, training=self.training)
            x = F.relu(self.fc2(x))
            x = self.log_softmax(x)
            return x
        
net = Net()

def evaluate(model, test_loader):
    model.eval()
    correct = 0 
    loss = 0
    with torch.no_grad():
        for data, labels in test_loader:
            data = data
            output = model(data)
            predicted = torch.max(output,1)[1]
            correct += (predicted == labels).sum()
            loss += F.nll_loss(output, labels).item()

    return (float(correct)/len(test)) *100, (loss/len(test_loader))

from tqdm import tqdm_notebook
def train_supervised(model, train_loader, test_loader):
    optimizer = torch.optim.SGD( model.parameters(), lr = 0.1)
    EPOCHS = 100
    model.train()
    for epoch in tqdm_notebook(range(EPOCHS)):
        correct = 0
        running_loss = 0
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            X_batch, y_batch = X_batch, y_batch
            
            output = model(X_batch)
            labeled_loss = F.nll_loss(output, y_batch)
                       
            optimizer.zero_grad()
            labeled_loss.backward()
            optimizer.step()
            running_loss += labeled_loss.item()
        
        if epoch %10 == 0:
            test_acc, test_loss = evaluate(model, test_loader)
            print('Epoch: {} : Train Loss : {:.5f} | Test Acc : {:.5f} | Test Loss : {:.3f} '.format(epoch, running_loss/(10 * len(train)), test_acc, test_loss))
            model.train()

train_supervised(net, train_loader, test_loader)




/home/ananyak/.local/lib/python3.6/site-packages/ipykernel_launcher.py:53: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


Epoch: 0 : Train Loss : 0.00737 | Test Acc : 9.58000 | Test Loss : 2.303 
Epoch: 10 : Train Loss : 0.00737 | Test Acc : 9.58000 | Test Loss : 2.302 
Epoch: 20 : Train Loss : 0.00734 | Test Acc : 25.61000 | Test Loss : 2.290 
Epoch: 30 : Train Loss : 0.00340 | Test Acc : 77.25000 | Test Loss : 0.735 
Epoch: 40 : Train Loss : 0.00148 | Test Acc : 89.66000 | Test Loss : 0.334 
Epoch: 50 : Train Loss : 0.00096 | Test Acc : 89.48000 | Test Loss : 0.326 
Epoch: 60 : Train Loss : 0.00069 | Test Acc : 92.75000 | Test Loss : 0.228 
Epoch: 70 : Train Loss : 0.00054 | Test Acc : 93.60000 | Test Loss : 0.202 
Epoch: 80 : Train Loss : 0.00040 | Test Acc : 93.72000 | Test Loss : 0.196 
Epoch: 90 : Train Loss : 0.00031 | Test Acc : 94.37000 | Test Loss : 0.187 



In [15]:
test_acc, test_loss = evaluate(net, test_loader)
print('Test Acc : {:.5f} | Test Loss : {:.3f} '.format(test_acc, test_loss))
torch.save(net.state_dict(), 'saved_models/supervised_weights')


Test Acc : 94.33000 | Test Loss : 0.194 


In [16]:
net.load_state_dict(torch.load('saved_models/supervised_weights'))

T1 = 100
T2 = 700
af = 3

def alpha_weight(epoch):
    if epoch < T1:
        return 0.0
    elif epoch > T2:
        return af
    else:
         return ((epoch-T1) / (T2-T1))*af
        
# Concept from : https://github.com/peimengsui/semi_supervised_mnist

from tqdm import tqdm_notebook

acc_scores = []
unlabel = []
pseudo_label = []

alpha_log = []
test_acc_log = []
test_loss_log = []
def semisup_train(model, train_loader, unlabeled_loader, test_loader):
    optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)
    EPOCHS = 150
    
    # Instead of using current epoch we use a "step" variable to calculate alpha_weight
    # This helps the model converge faster
    step = 100 
    
    model.train()
    for epoch in tqdm_notebook(range(EPOCHS)):
        for batch_idx, x_unlabeled in enumerate(unlabeled_loader):
            
            
            # Forward Pass to get the pseudo labels
            x_unlabeled = x_unlabeled[0]
            model.eval()
            output_unlabeled = model(x_unlabeled)
            _, pseudo_labeled = torch.max(output_unlabeled, 1)
            model.train()
            
            
            """ ONLY FOR VISUALIZATION"""
            if (batch_idx < 3) and (epoch % 10 == 0):
                unlabel.append(x_unlabeled.cpu())
                pseudo_label.append(pseudo_labeled.cpu())
            """ ********************** """
            
            # Now calculate the unlabeled loss using the pseudo label
            output = model(x_unlabeled)
            unlabeled_loss = alpha_weight(step) * F.nll_loss(output, pseudo_labeled)   
            
            # Backpropogate
            optimizer.zero_grad()
            unlabeled_loss.backward()
            optimizer.step()
            
            
            # For every 50 batches train one epoch on labeled data 
            if batch_idx % 50 == 0:
                
                # Normal training procedure
                for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
                    X_batch = X_batch
                    y_batch = y_batch
                    output = model(X_batch)
                    labeled_loss = F.nll_loss(output, y_batch)

                    optimizer.zero_grad()
                    labeled_loss.backward()
                    optimizer.step()
                
                # Now we increment step by 1
                step += 1
                

        test_acc, test_loss =evaluate(model, test_loader)
        print('Epoch: {} : Alpha Weight : {:.5f} | Test Acc : {:.5f} | Test Loss : {:.3f} '.format(epoch, alpha_weight(step), test_acc, test_loss))
        
        """ LOGGING VALUES """
        alpha_log.append(alpha_weight(step))
        test_acc_log.append(test_acc/100)
        test_loss_log.append(test_loss)
        """ ************** """
        model.train()
        
semisup_train(net, train_loader, unlabeled_loader, test_loader)


/home/ananyak/.local/lib/python3.6/site-packages/ipykernel_launcher.py:35: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


Epoch: 0 : Alpha Weight : 0.02500 | Test Acc : 94.22000 | Test Loss : 0.200 
Epoch: 1 : Alpha Weight : 0.05000 | Test Acc : 94.76000 | Test Loss : 0.191 
Epoch: 2 : Alpha Weight : 0.07500 | Test Acc : 94.86000 | Test Loss : 0.187 
Epoch: 3 : Alpha Weight : 0.10000 | Test Acc : 94.78000 | Test Loss : 0.189 
Epoch: 4 : Alpha Weight : 0.12500 | Test Acc : 94.77000 | Test Loss : 0.186 
Epoch: 5 : Alpha Weight : 0.15000 | Test Acc : 94.92000 | Test Loss : 0.186 
Epoch: 6 : Alpha Weight : 0.17500 | Test Acc : 95.08000 | Test Loss : 0.177 
Epoch: 7 : Alpha Weight : 0.20000 | Test Acc : 95.26000 | Test Loss : 0.176 
Epoch: 8 : Alpha Weight : 0.22500 | Test Acc : 95.27000 | Test Loss : 0.176 
Epoch: 9 : Alpha Weight : 0.25000 | Test Acc : 95.29000 | Test Loss : 0.172 
Epoch: 10 : Alpha Weight : 0.27500 | Test Acc : 95.51000 | Test Loss : 0.167 
Epoch: 11 : Alpha Weight : 0.30000 | Test Acc : 95.60000 | Test Loss : 0.161 
Epoch: 12 : Alpha Weight : 0.32500 | Test Acc : 95.60000 | Test Loss : 0.1

Epoch: 106 : Alpha Weight : 2.67500 | Test Acc : 98.00000 | Test Loss : 0.087 
Epoch: 107 : Alpha Weight : 2.70000 | Test Acc : 98.09000 | Test Loss : 0.086 
Epoch: 108 : Alpha Weight : 2.72500 | Test Acc : 98.20000 | Test Loss : 0.081 
Epoch: 109 : Alpha Weight : 2.75000 | Test Acc : 98.11000 | Test Loss : 0.083 
Epoch: 110 : Alpha Weight : 2.77500 | Test Acc : 97.99000 | Test Loss : 0.088 
Epoch: 111 : Alpha Weight : 2.80000 | Test Acc : 97.95000 | Test Loss : 0.091 
Epoch: 112 : Alpha Weight : 2.82500 | Test Acc : 98.10000 | Test Loss : 0.088 
Epoch: 113 : Alpha Weight : 2.85000 | Test Acc : 98.13000 | Test Loss : 0.086 
Epoch: 114 : Alpha Weight : 2.87500 | Test Acc : 98.04000 | Test Loss : 0.087 
Epoch: 115 : Alpha Weight : 2.90000 | Test Acc : 98.10000 | Test Loss : 0.085 
Epoch: 116 : Alpha Weight : 2.92500 | Test Acc : 98.26000 | Test Loss : 0.081 
Epoch: 117 : Alpha Weight : 2.95000 | Test Acc : 98.10000 | Test Loss : 0.088 
Epoch: 118 : Alpha Weight : 2.97500 | Test Acc : 98.

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_b

KeyboardInterrupt: 